## CausalML

In [2]:
import pandas as pd
import numpy as np
import networkx as nx
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor
from xgboost import XGBRegressor
import warnings

from causalml.inference.meta import LRSRegressor
from causalml.inference.meta import XGBTRegressor, MLPTRegressor
from causalml.inference.meta import BaseXRegressor
from causalml.inference.meta.tmle import TMLELearner
from causalml.match import NearestNeighborMatch, MatchOptimizer, create_table_one
from causalml.propensity import ElasticNetPropensityModel
from causalml.dataset import *
from causalml.metrics import *
from causalml.inference.tree import CausalRandomForestRegressor


from causallearn.search.FCMBased import lingam
from causallearn.search.ConstraintBased.PC import pc
from causallearn.utils.cit import chisq
from dowhy import CausalModel
from dowhy.utils.graph_operations import adjacency_matrix_to_graph

warnings.filterwarnings('ignore')
plt.style.use('fivethirtyeight')


In [4]:
y, X, treatment, tau, b, e = synthetic_data(mode=4, n=10000, p=8, sigma=1.0)
print("True ATE: {:.03f}".format(tau.mean()))
print(help(synthetic_data))

True ATE: 0.121
Help on function synthetic_data in module causalml.dataset.regression:

synthetic_data(mode=1, n=1000, p=5, sigma=1.0, adj=0.0)
    Synthetic data in Nie X. and Wager S. (2018) 'Quasi-Oracle Estimation of Heterogeneous Treatment Effects'
    Args:
        mode (int, optional): mode of the simulation:             1 for difficult nuisance components and an easy treatment effect.             2 for a randomized trial.             3 for an easy propensity and a difficult baseline.             4 for unrelated treatment and control groups.             5 for a hidden confounder biasing treatment.
        n (int, optional): number of observations
        p (int optional): number of covariates (>=5)
        sigma (float): standard deviation of the error term
        adj (float): adjustment term for the distribution of propensity, e. Higher values shift the distribution to 0.
                     It does not apply to mode == 2 or 3.
    Returns:
        (tuple): Synthetically gene

#### X-Learner

In [3]:

learner_x = BaseXRegressor(learner=XGBRegressor())
ate_x = learner_x.estimate_ate(X=X, treatment=treatment, y=y, p=e)
print('Using XGB:')
print(ate_x)

print('ATE estimate: {:.03f}'.format(ate_x[0][0]))
print('ATE lower bound: {:.03f}'.format(ate_x[1][0]))
print('ATE upper bound: {:.03f}'.format(ate_x[2][0]))

print()

learner_x = BaseXRegressor(learner=LinearRegression())
ate_x = learner_x.estimate_ate(X=X, treatment=treatment, y=y, p=e)
print('Using Linear Regression:')
print('ATE estimate: {:.03f}'.format(ate_x[0][0]))
print('ATE lower bound: {:.03f}'.format(ate_x[1][0]))
print('ATE upper bound: {:.03f}'.format(ate_x[2][0]))


Using XGB:
(array([0.14295317]), array([0.11201952]), array([0.17388681]))
ATE estimate: 0.143
ATE lower bound: 0.112
ATE upper bound: 0.174

Using Linear Regression:
ATE estimate: 0.068
ATE lower bound: 0.015
ATE upper bound: 0.120


In [4]:
rf = RandomForestRegressor(
    n_estimators=400,      
    max_depth=14,         
    min_samples_split=80,  
    min_samples_leaf=30,   
    max_features="sqrt", 
    bootstrap=True,
    oob_score=True, 
    n_jobs=-1,
    random_state=0,
)

learner_x = BaseXRegressor(learner=rf)
ate_x = learner_x.estimate_ate(X=X, treatment=treatment, y=y, p=e)
print('Using Random Forest:')
print('ATE estimate: {:.03f}'.format(ate_x[0][0]))
print('ATE lower bound: {:.03f}'.format(ate_x[1][0]))
print('ATE upper bound: {:.03f}'.format(ate_x[2][0]))


Using Random Forest:
ATE estimate: 0.238
ATE lower bound: 0.193
ATE upper bound: 0.284


In [5]:
mlp_base = Pipeline([
    ("scaler", StandardScaler()),
    ("mlp", MLPRegressor(
        hidden_layer_sizes=(256, 128, 64),
        activation="relu",
        alpha=5e-5,
        learning_rate_init=1e-3,
        batch_size=256,
        max_iter=600,
        early_stopping=True,
        validation_fraction=0.15,
        n_iter_no_change=12,
        tol=1e-4,
        random_state=42

    ))
])


learner_x = BaseXRegressor(learner=mlp_base)
ate_x = learner_x.estimate_ate(X=X, treatment=treatment, y=y, p=e)
print('Using MLP:')
print('ATE estimate: {:.03f}'.format(ate_x[0][0]))
print('ATE lower bound: {:.03f}'.format(ate_x[1][0]))
print('ATE upper bound: {:.03f}'.format(ate_x[2][0]))

Using MLP:
ATE estimate: 0.096
ATE lower bound: 0.048
ATE upper bound: 0.144


#### Causal Forest

In [6]:
n=10000

crf = CausalRandomForestRegressor(
    n_estimators=500,
    max_depth=16,
    min_samples_split=100,
    min_samples_leaf=40,
    max_features="sqrt",
    bootstrap=True, 
    oob_score=False,
    random_state=42,
    n_jobs=-1
)
crf.fit(X, treatment, y)

ite = crf.predict(X)
ate = np.mean(ite)


KeyboardInterrupt: 

In [ ]:
ates = []
n_boot = 100
rng = np.random.RandomState(42)
for _ in range(n_boot):
    idx = rng.choice(np.arange(n), size=n, replace=True)
    ite_b = crf.predict(X[idx])
    ates.append(np.mean(ite_b))
ates = np.array(ates)

ci_lower, ci_upper = np.percentile(ates, [2.5, 97.5])

print("ATE estimate: {:.03f}".format(ate))
print("ATE lower bound: {:.03f}".format(ci_lower))
print("ATE upper bound: {:.03f}".format(ci_upper))

KeyboardInterrupt: 

#### TMLE

In [7]:
pm = ElasticNetPropensityModel(n_fold=5, random_state=42)
p_hat = pm.fit_predict(X, treatment)

tmle = TMLELearner(learner=GradientBoostingRegressor(random_state=42))
ate_tmle = tmle.estimate_ate(X, treatment=treatment, y=y, p=p_hat, return_ci=True)

print('Using TMLE:')
print('ATE estimate: {:.03f}'.format(ate_tmle[0][0]))
print('ATE lower bound: {:.03f}'.format(ate_tmle[1][0]))
print('ATE upper bound: {:.03f}'.format(ate_tmle[2][0]))

Using TMLE:
ATE estimate: 0.154
ATE lower bound: 0.101
ATE upper bound: 0.207


## DoWhy + Causal_Learn

In [8]:
cols = [f"X{i+1}" for i in range(X.shape[1])] + ["T", "Y"]
df = pd.DataFrame(np.column_stack([X, treatment, y]), columns=cols)

In [9]:
model = lingam.ICALiNGAM()
model.fit(df.values)
A = model.adjacency_matrix_

labels = list(df.columns)
G = nx.DiGraph()
for i in range(len(labels)):
    for j in range(len(labels)):
        if A[i, j] != 0:
            G.add_edge(labels[i], labels[j])


causal_model = CausalModel(data=df, treatment="T", outcome="Y", graph=G)
identified = causal_model.identify_effect()
estimate = causal_model.estimate_effect(
    identified,
    method_name="backdoor.propensity_score_weighting"
)

print("LiNGAM → DoWhy ATE (IPW):", estimate.value)


LiNGAM → DoWhy ATE (IPW): 0
